<a href="https://colab.research.google.com/github/AnjaliRajagopal/Algorithmofmassivedata/blob/main/AMD_Anjali_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing Packages and Environment Setup

In [ ]:
! pip install kaggle spark-nlp==3.4.2 pyspark==3.2.0 ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 4.2 MB/s 
     |████████████████████████████████| 281.3 MB 30 kB/s 
     |████████████████████████████████| 198 kB 50.2 MB/s 
     |████████████████████████████████| 1.6 MB 40.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=56efc8f1d6fd415997c8f4d027f8861e2e526b5d45f60f4f083922e50497e64c
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
time: 580 µs (started: 2022-09-15 18:54:40 +00:00)


In [ ]:
from pyspark.sql import SparkSession
import os
from glob import glob
from pyspark.sql.types import FloatType
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import * 
import string
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas as pd
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from itertools import combinations
import math
import numpy as np
import logging
from joblib import Parallel, delayed, cpu_count
from collections import namedtuple
from itertools import product
import time
from itertools import chain
spark = sparknlp.start(spark32 = True)
sc = spark.sparkContext


time: 31.5 s (started: 2022-09-15 18:55:10 +00:00)


# Data Setup

## Data Download
The first step is to obtain the dataset for "Ukraine Conflict Twitter". The direct download option through the command line is available on Kaggle's website. The command kaggle datasets download, in particular, requires the precise name of the item to be downloaded and the person's login credentials.

In [ ]:
# The below code can be used to download data from Kaggle into the Direcotry ad /content/Kaggle:


path = '/content/Kaggle'

if not os.path.exists(path):
  # Create a new directory because it does not exist 
  os.makedirs(path)

%cd /content/Kaggle/


os.environ['KAGGLE_USERNAME'] = "anjalirajagopal" # username from the json file
os.environ['KAGGLE_KEY'] = "dd4b25d5edbb7a9dbffd6713ddde2808" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/Kaggle"


! kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows 
! unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

/content/Kaggle
100% 11.9G/11.9G [01:53<00:00, 152MB/s]
100% 11.9G/11.9G [01:53<00:00, 113MB/s]
Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCom

# Data File Extension Change

In order to read the downloaded data directly into PySpark, we needed to change the file extension of the downloaded files from ".gzip" to ".gz". The below code performs this operation.

In [ ]:
#files = glob('./*.gzip')
files = glob('/content/Kaggle/UkraineWar/UkraineWar/*.gzip')
# files = glob('./Kaggle/UkraineWar/UkraineWar/*.gzip')
for file in files:
  os.rename(file, '.'.join(file.split('.')[0:2]+['gz']))

time: 9.07 ms (started: 2022-09-15 19:01:49 +00:00)


In [ ]:
# Selecting file and sampling data (using only 0.5% of one day's data)
df = (
    spark
    .read
    .option("multiLine", 'true')
    .option("escape", "\"")
    .csv("/content/Kaggle/UkraineWar/UkraineWar/0401_UkraineCombinedTweetsDeduped.csv.gz", header=True)
    .select(['userid','language','retweetcount','text'])) 

df.show(5)

+-------------------+--------+------------+--------------------+
|             userid|language|retweetcount|                text|
+-------------------+--------+------------+--------------------+
|           16882774|      en|        3412|⚡The Ukrainian Ai...|
|         3205296069|      en|         100|Chernihiv oblast....|
|1235940869812809728|      en|           9|America 🇺🇸 is p...|
|1347985375566966784|      en|         573|JUST IN: #Anonymo...|
|1505394816636846083|      en|         190|***PUBLIC MINT NO...|
+-------------------+--------+------------+--------------------+
only showing top 5 rows

time: 9.5 s (started: 2022-09-15 19:01:55 +00:00)


# Data Filtering



In [ ]:
# Filtering data which is only in English language, lies in the above 50% quantile range of retweet count, should be unique as per user id and text.  

df = df.withColumn("retweetcount", df["retweetcount"].cast(FloatType()))
only_text = (
            df
            .dropna(subset = ['text','language'])
            .filter((df.language == "en") & (df.retweetcount >= df.approxQuantile("retweetcount", [0.5], 0.25)[0])) 
            .dropDuplicates(["userid","text"])
            .select("text")
            .sample(fraction = 0.005,seed = 2022)) # Change the Fraction Argument to take different sample sizes

only_text.show(5)

+--------------------+
|                text|
+--------------------+
|#Russia #Ukraine ...|
|#Mariupol April 1...|
|⚡The Ukrainian Ai...|
|My government dec...|
|Courage, strength...|
+--------------------+
only showing top 5 rows

time: 12.5 s (started: 2022-09-15 19:02:16 +00:00)


#Text preprocessing




In [ ]:
# Download lemmatizer dependencies
!wget -q https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

# Loading 'text' data into spark NLP  
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Tokenization: Converting single words into individual tokens
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normaliszation: Removing numbers, punctuations and other non-alphabet characters
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

# Lemmatization: Converting the words to their root forms
lemmatizer = Lemmatizer() \
    .setInputCols(["normalized"]) \
    .setOutputCol("lemma") \
    .setDictionary("/content/Kaggle/AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

# Clean Stopwords: Removing the stopwords which are by-default loaded in the stopwords cleaner library
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("lemma")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)
      # .setStopWords(["no", "without"])
stopwordsList = stopwords_cleaner.getStopWords()
custom_stopword_list = ['ukraine','russia','ukrainian','russian'] # add a list of customized stop words here
stopwordsList.extend(custom_stopword_list)
stopwords_cleaner = stopwords_cleaner.setStopWords(stopwordsList)

# Generating ngrams: Identifying a set of words ('n' no of words) used together
ngrams_cum = NGramGenerator().setInputCols(["cleanTokens"]) \
      .setOutputCol("ngrams") \
      .setN(1)\
      .setEnableCumulative(False)\
      .setDelimiter("_") 

# Providing the output in the finished format
finisher = Finisher() \
    .setInputCols(["ngrams"]) \
    .setIncludeMetadata(False)

# Setting up the sequence of execution
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    tokenizer,
    normalizer,
    lemmatizer,
    stopwords_cleaner,
    ngrams_cum,
    finisher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
result = pipelineModel.transform(only_text)

result = result.withColumn("unique", F.array_distinct("finished_ngrams"))

# Filtering data based on word count (word count should be greated than 1 and less than 75% of distribution (to avoid outlier and extremly lenghthy tweets))

slen = udf(lambda s: len(s), IntegerType())
result = result.withColumn("word_count", slen(result.unique))
result = result.filter((result.word_count > 1) & (result.word_count < result.approxQuantile("word_count", [0.75], 0.25)[0]))
result.show(5)
transactionList = result.select('unique').rdd.flatMap(lambda x: x)


+--------------------+--------------------+--------------------+----------+
|                text|     finished_ngrams|              unique|word_count|
+--------------------+--------------------+--------------------+----------+
|#Russia #Ukraine ...|[recent, photo, f...|[recent, photo, f...|        23|
|#Mariupol April 1...|[mariupol, april,...|[mariupol, april,...|         5|
|⚡The Ukrainian Ai...|[air, force, like...|[air, force, like...|        19|
|My government dec...|[government, deci...|[government, deci...|        19|
|Courage, strength...|[courage, strengt...|[courage, strengt...|         9|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

time: 26.3 s (started: 2022-09-15 19:03:01 +00:00)


#Apriori Algorithm

In [ ]:
def applyApriori(transactionList, max_itemset_size, minSupport, min_conf):
    transactionCount = transactionList.count()
    minSupport = math.floor(minSupport * len(transactionList.collect()))
    indexDictionary, reverse_index_dictionary = generate_dictionary_from_items(transactionList=transactionList)
    indexedTransactions = transactionList.map(
        lambda x: encode_items(x, indexDictionary)
    )
    indexedItems = transactionList.flatMap(lambda x: encode_items(x, indexDictionary))

    frequentItemsTable = np.zeros(len(indexDictionary))
    count_frequent_items = filter_by_min_count(indexedItems, minSupport)
    frequentItemsTable = update_frequent_items_table(
        count_frequent_items, len(indexDictionary)
    )

    i = 1
    results = None
    while i <= max_itemset_size:
        frequent_itemset = get_frequent_item_sets(
            indexedTransactions, i, minSupport, frequentItemsTable
        )
        frequentItemsTable = update_frequent_items_table(
            frequent_itemset, len(indexDictionary)
        )
        if results is None:
            results = frequent_itemset
        else:
            results = sc.union([results, frequent_itemset])
        i += 1
    results_decoded = results.map(lambda x: (tuple(decode_items(x[0], reverse_index_dictionary)),x[1]))
    results_decoded_collected = results_decoded.collect()
    rules = Parallel(n_jobs=cpu_count() - 1)(delayed(generate_rules_from_itemset)(result,results_decoded_collected,
                                                                                  min_conf,
                                                                                  transactionCount) for result in results_decoded_collected)
    rules = [r for r in rules if r != []]
    return rules, results_decoded

time: 6.25 ms (started: 2022-09-15 19:03:31 +00:00)


# Helper Functions

## Generate Dictionary From Items

This function creates a numeric encoding for all unique items present in the provided list of transactions. This numeric encoding is primarily used for efficient operations as it reduces the memory footprint of the collections.

In [ ]:
def generate_dictionary_from_items(transactionList):
    uniqueTokens = transactionList.flatMap(lambda x: x).distinct()
    dictionary = dict(zip(uniqueTokens.collect(), range(len(uniqueTokens.collect()))))
    reverse_dictionary = {v: k for k, v in dictionary.items()}
    return dictionary, reverse_dictionary

time: 1.46 ms (started: 2022-09-15 19:03:36 +00:00)


## Encode Items

This function encodes a collection of items to their numeric counterparts as defined in the dictionary created using the function (generate_dictionary_from_items)

In [ ]:
def encode_items(items, dictionary):
    return [dictionary[item] for item in items]

time: 1.08 ms (started: 2022-09-15 19:03:38 +00:00)


## Decode Items

This function decodes a collection of encoded items to their string counterparts as defined in the dictionary created using the function (generate_dictionary_from_items)

In [ ]:
def decode_items(encoded_items, dictionary):

    return [dictionary[encoded_item] for encoded_item in encoded_items]

time: 1.1 ms (started: 2022-09-15 19:03:49 +00:00)


## Filter By Min Count

This function calculates frequency counts for items in a PySpark RDD and filters the RDD using a minimum count threshold

In [ ]:
def filter_by_min_count(items, minCountThresh):
    filtered_items = (
        items.map(lambda x: (x, 1))
        .reduceByKey(lambda a, b: a + b)
        .filter(lambda item: item[1] >= minCountThresh)
    )
    return filtered_items

time: 1.5 ms (started: 2022-09-15 19:03:57 +00:00)


## Update Frequent Items Table

This function creates a 1-dimensional numpy array with a size equal to the number of unique items (based on the index dictionary created using (generate_dictionary_from_items)) with item indexes (based on the index dictionary created using (generate_dictionary_from_items)) as the index and assigns a numeric value as the value. The numeric value is set to 0 for an item if the item is not considered to be frequent.

In [ ]:
def update_frequent_items_table(frequent_items, index_dictionary_size):
    frequent_itemsets = np.zeros(index_dictionary_size)
    i = 0
    for itemset in frequent_items.collect():
        if isinstance(itemset[0], tuple):
            for item in itemset[0]:
                i = i + 1
                frequent_itemsets[item] = i
        else:
            i = i + 1
            frequent_itemsets[itemset[0]] = i
    return frequent_itemsets

time: 1.29 ms (started: 2022-09-15 19:04:03 +00:00)


## Get Frequent Item Sets


In [ ]:
def get_frequent_item_sets(
    transactionList, itemset_size, minSupport, frequent_tokens_table
):
    frequent_itemsets = (
        transactionList.map(lambda x: filter_frequent(x, frequent_tokens_table))
        .filter(lambda x: len(x) > 1)
        .flatMap(lambda x: combinations(x, itemset_size))
        .map(lambda x: (x, 1))
        .reduceByKey(lambda x1, x2: x1 + x2)
        .filter(lambda x: x[1] >= minSupport)
    )
    return frequent_itemsets

time: 1.37 ms (started: 2022-09-15 19:04:08 +00:00)


## Filter Frequent

This function filters an iterable, keeping only the items that are considered frequent, as per the array created using (update_frequent_items_table)
.

In [ ]:
def filter_frequent(items, frequent_items_table):
    return [item for item in items if frequent_items_table[item] != 0]

time: 1.14 ms (started: 2022-09-15 19:04:11 +00:00)


## Generate rules from itemset



In [ ]:
def generate_rules_from_itemset(itemset, frequent_items, min_conf, transaction_count):
    
    rules = []
    k = len(itemset[0])
    if k >= 2:
        subsets = list(combinations(itemset[0], k - 1))
        support = itemset[1]
        for antecedent in subsets:
            antecedent_tuple = [items for items in frequent_items if items[0] == antecedent][0]
            antecedent_support = antecedent_tuple[1]
            confidence = float("{0:.2f}".format(support / antecedent_support))
            if confidence >= min_conf:
                consequent = tuple(
                    filter(lambda x: x not in antecedent, itemset[0])
                )
                rule = {
                    "antecedent": antecedent,
                    "consequent": consequent,
                    "confidence": confidence,
                    "support": float("{0:.3f}".format(support / transaction_count)),
                }
                rules.append(rule)

    return rules

time: 3.6 ms (started: 2022-09-15 19:04:17 +00:00)


# Experiments

In this section, I applied the Apriori Algorithm implementation I have developed to the dataset as described in the sections before. The code below tests out the algorithm in a variety of parameter combinations. It takes 3 separate minimum support values, 3 separate minimum confidence values, and 2 different max item sizes to evaluate the drivers of performance. Subsequently, I also ran a comparison between my implementation of the Apriori algorithm and the highly scalable FP-Growth implementation included with PySpark. 

## Apriori Algorithm with key performance drivers

In [ ]:

supports_list = [0.01,0.025,0.05]
conf_list = [0.05,0.1,0.2]
itemset_size_list = [2,3]


times = []
n_itemsets = []
n_rules = []
params = list(product(supports_list, conf_list, itemset_size_list))

def run_experiment(param,transactionList):
  result = namedtuple('result','params time n_itemsets n_rules')
  start = time.time()
  rules, itemsets = applyApriori(transactionList = transactionList,
                                max_itemset_size = param[2], 
                                minSupport = param[0], 
                                min_conf = param[1])
  end = time.time()
  times = end-start
  n_itemsets = len(itemsets.collect())
  n_rules = len(rules)
  return result(param, times, n_itemsets, n_rules)


results = Parallel(n_jobs=cpu_count() - 1)(delayed(run_experiment)(param, transactionList) for param in params)

df = pd.DataFrame.from_records(
   results,
   columns=['params','time','n_itemsets', 'n_rules']
)

df['minSupport'], df['minConfidence'], df['maxItemSets'] = zip(*df.params)
df = df[['minSupport','minConfidence','maxItemSets','time','n_itemsets','n_rules']]
df.to_csv('Apriori Performance Drive Experiment.csv', index = False)
df

,minSupport,minConfidence,maxItemSets,time,n_itemsets,n_rules
0,0.010,0.05,2,17.330990,2974,2517
1,0.010,0.05,3,62.174168,15041,14584
2,0.010,0.10,2,10.685942,2974,2516
3,0.010,0.10,3,59.609496,15041,14583
4,0.010,0.20,2,9.781859,2974,2498
5,0.010,0.20,3,60.183365,15041,14565
6,0.025,0.05,2,7.714690,390,248
7,0.025,0.05,3,10.084576,1456,1314
8,0.025,0.10,2,7.621615,390,248
9,0.025,0.10,3,9.866850,1456,1314


time: 5min 26s (started: 2022-09-15 19:04:23 +00:00)


## Apriori vs FP Growth Performance

In this experiment I run both the Apriori and FP Growth algorithms with Maximum Itemset Size = 2, Minimum Support = 0.05 and Minimum Confidence = 0.05 and compare their performances.

## Apriori

In [ ]:

apriori_experiment_1_rules, apriori_experiment_1_itemsets,  = applyApriori(transactionList = transactionList,
                                                                            max_itemset_size = 2, 
                                                                            minSupport = 0.05, 
                                                                            min_conf = 0.05)
apriori_experiment_1_rules = pd.DataFrame(list(chain.from_iterable(apriori_experiment_1_rules)))
apriori_experiment_1_rules.to_csv('apriori_experiment_1_rules.csv',index = False)


time: 7.43 s (started: 2022-09-15 19:09:58 +00:00)


## FP Growth

In [ ]:
fpGrowth = FPGrowth(itemsCol="unique", minSupport=0.05, minConfidence=0.05)
model = fpGrowth.fit(result)

items = model.freqItemsets
rules = model.associationRules
rules.show(10)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------------+----------+------------------+--------------------+
|          antecedent|  consequent|confidence|              lift|             support|
+--------------------+------------+----------+------------------+--------------------+
|[misinformation, ...|    [regard]|       1.0|19.363636363636363|0.051643192488262914|
|[misinformation, ...|     [force]|       1.0| 5.916666666666667|0.051643192488262914|
|[misinformation, ...|[stoprussia]|       1.0|11.410714285714285|0.051643192488262914|
|[misinformation, ...|      [nato]|       1.0|11.210526315789474|0.051643192488262914|
|[misinformation, ...| [situation]|       1.0|13.891304347826088|0.051643192488262914|
|[misinformation, ...|   [western]|       1.0| 17.27027027027027|0.051643192488262914|
|[misinformation, ...|       [sky]|       1.0|18.257142857142856|0.051643192488262914|
|[misinformation, ...|   [publish]|       1.0|18.257142857142856|0.051643192488262914|
|[misinformation, ...|    [outlet]|       1